In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline
# This blokc is important if we want the memory to grow on the GPU, and not block allocate the whole thing
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from datetime import datetime
import os
from pathlib import Path
from sklearn.utils import class_weight 
from numpy import vstack, asarray
from keras.optimizers import Adam, Nadam
import pandas as pd

# Hyperparam opt
import talos as ta

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

# Set path to find modelling tools for later use
import sys
sys.path.append(os.path.join(os.getcwd(),".."))
# Global params live here
import haberrspd.charCNN.globals
from haberrspd.charCNN.models_tf import char_cnn_model_talos
from haberrspd.charCNN.data_utilities import create_training_data_keras


Using TensorFlow backend.


## Assign a loss function

In [ ]:
loss_func = 'squared_hinge'

if loss_func == 'hinge' or loss_func == 'squared_hinge':
    y_train = [-1 if x==0 else x for x in y_train]
    y_test = [-1 if x==0 else x for x in y_test]
    
if loss_func == 'binary_crossentropy':
    # Check if label-space is correct
    if (-1 in y_train) or (-1 in y_test):
        y_train = [0 if x==-1 else x for x in y_train]
        y_test = [0 if x==-1 else x for x in y_test]

## Train model

In [ ]:
# Compile
model.compile(loss=loss_func,  # TODO: change to cosine loss, cosine_proximity, binary_crossentropy
              optimizer='adam',            # TODO: check which is most appropriate
              metrics=['accuracy'])        # Probs other options here which are more useful

# Check if checkpoints dir exists, if not make it
if not os.path.exists('../../keras_checkpoints'):
    os.makedirs('../../keras_checkpoints')

# Callbacks
file_name = "char-CNN"
check_cb = ModelCheckpoint(file_name + '.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=0,
                           save_best_only=True,
                           mode='min')

earlystop_cb = EarlyStopping(monitor='val_loss',
                             patience=7,
                             verbose=0,
                             mode='auto')

# history = LossHistory()
"""
TODO:

-Add class-weight option to take into account class-imbalance on patients and controls
"""
fit_hist = model.fit(X_train,
                     y_train,
                     validation_data=(X_test, y_test),
                     verbose=0, # Set to zero if using live plotting of losses
                     class_weight = class_weights,
                     batch_size=128,
                     epochs=40,
                     #shuffle=True, # Our data is already shuffled during data loading
                     callbacks=[
                                #check_cb,
                                #tensorboard_callback,
                                PlotLossesCallback(),
                                #earlystop_cb
                               ]
                    )

# TALOS: hyperparameter optimisation

In [2]:
from haberrspd.charCNN.data_utilities import (us_keyboard_keys_to_2d_coordinates_mrc, 
                                              english_language_qwerty_keyboard, 
                                              uk_standard_layout_keyboard, 
                                              uk_keyboard_keys_to_2d_coordinates_mjff,
                                              uk_keyboard_keys_to_2d_coordinates_mjff_old)
from haberrspd.preprocess import modifier_key_replacements
import numpy as np
import itertools

----

### MJFF

In [3]:
keyboard_lower, keyboard_upper = english_language_qwerty_keyboard(layout="uk")

In [6]:
test_sent = 'i am a little pony ^="ω"'

In [12]:
uk_keyboard_keys_to_2d_coordinates_mjff_old(test_sent,keyboard)

array([[ 1,  8],
       [ 4,  3],
       [ 2,  1],
       [ 3,  7],
       [ 4,  3],
       [ 2,  1],
       [ 4,  3],
       [ 2,  9],
       [ 1,  8],
       [ 1,  5],
       [ 1,  5],
       [ 2,  9],
       [ 1,  3],
       [ 4,  3],
       [ 1, 10],
       [ 1,  9],
       [ 3,  6],
       [ 1,  6],
       [ 4,  3],
       [ 5,  6]])

In [7]:
uk_keyboard_keys_to_2d_coordinates_mjff(test_sent,keyboard_lower,keyboard_upper)

array([[ 1,  8],
       [ 4,  3],
       [ 2,  1],
       [ 3,  7],
       [ 4,  3],
       [ 2,  1],
       [ 4,  3],
       [ 2,  9],
       [ 1,  8],
       [ 1,  5],
       [ 1,  5],
       [ 2,  9],
       [ 1,  3],
       [ 4,  3],
       [ 1, 10],
       [ 1,  9],
       [ 3,  6],
       [ 1,  6],
       [ 4,  3],
       [ 0,  6],
       [ 0, 12],
       [ 0,  2],
       [ 3,  7],
       [ 0,  2]])

---

### MRC

In [3]:
df = pd.read_pickle(Path("../data/MRC") / 'processed_mrc_dataset.pkl')

In [27]:
df.head()

,key,type,location,timestamp,participant_id,sentence_id,diagnosis,medication
0,β,keydown,1,25885.055,1010,1,0,0
1,h,keydown,0,26086.840,1010,1,0,0
2,β,keyup,1,26181.975,1010,1,0,0
3,h,keyup,0,26193.745,1010,1,0,0
4,o,keydown,0,26321.480,1010,1,0,0


In [13]:
locs = df[(df.participant_id == 1010) & (df.sentence_id == 1)].location.values
sent = df[(df.participant_id == 1010) & (df.sentence_id == 1)].key.tolist()

In [20]:
locs[:5]

array([1, 0, 1, 0, 0])

In [26]:
list(itertools.chain.from_iterable([[i]*int(n) for i,n in zip(locs[:5][:-1],[4,3,2,2,4][1:])]))

[1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]

In [10]:
out = us_keyboard_keys_to_2d_coordinates_mrc(sent,locs,us_lower,us_upper)

In [ ]:
DATA_ROOT = Path("../data/") / "MJFF" / "preproc"
X_train, X_test, y_train, y_test, max_sentence_length, alphabet_size = create_training_data_keras(
    DATA_ROOT, "char", "EnglishData-preprocessed_attempt_1.csv")

# Class weights are dynamic as the data-loader is stochastic and changes with each run.
class_weights = dict(zip([0, 1], class_weight.compute_class_weight("balanced", list(set(y_train)), y_train)))

### Small

In [4]:
params ={
         # Learning rate
         'lr': (0.1, 10, 5),
         'conv_output_space' : [4],#,8],
         'number_of_large_filters' : [2],             
         'number_of_small_filters' : [2],
         'large_filter_length' : [60],
         'small_filter_length' : [5],
         'pool_length' : [2,4,8],
         'dense_units_layer_3' : [32],
         'dense_units_layer_2' : [16],
         'batch_size': [32],
         'epochs': [100],
         'dropout': [0.05,0.1,0.2],
         'conv_kernel_initializer': ['uniform'],
         'conv_bias_initializer': ['uniform'],
         'dense_kernel_initializer': ['uniform'],
         'dense_bias_initializer': ['uniform'],
         'optimizer': [Adam],
         'loss': ['binary_crossentropy'],
         'conv_activation':['relu'],
         'dense_activation':['relu'],
         'last_activation': ['sigmoid'],
         # Stationary parameters, i.e. do not get optimised
         'max_sentence_length':[max_sentence_length],
         'alphabet_size':[alphabet_size],
         'control_class_weight' : [class_weights[0]],
         'pd_class_weight' : [class_weights[1]],
            }

Total number of characters: 49
{0: 0.7840059790732437, 1: 1.3802631578947369}


In [6]:
opt_params ={
            'lr': (0.1, 10, 2),  # Learning rate
            'conv_output_space': [4, 8, 16],  # ,8],
            'number_of_large_filters': [2],
            'number_of_small_filters': [2],
            'large_filter_length': [20, 40, 80, 160],
            'small_filter_length': [5, 10, 20],
            'pool_length': [2],
            'dense_units_layer_3': [32],
            'dense_units_layer_2': [16],
            'batch_size': [16, 32, 64],
            'epochs': [200],
            'dropout': (0, 0.5, 2),
            'conv_kernel_initializer': ['uniform'],
            'conv_bias_initializer': ['uniform'],
            'dense_kernel_initializer': ['uniform'],
            'dense_bias_initializer': ['uniform'],
            'optimizer': [Nadam],  # If used this way, these have to explicitly imported
            'loss': ['binary_crossentropy'],  # Loss functions
            'conv_activation': ['relu'],
            'dense_activation': ['relu'],
            'last_activation': ['sigmoid'],
            # Stationary parameters, i.e. do not get optimised
            'max_sentence_length': [max_sentence_length],
            'alphabet_size': [alphabet_size],
            'control_class_weight': [class_weights[0]],
            'pd_class_weight': [class_weights[1]],
            }

"""
'sgd': SGD,
'rmsprop': RMSprop,
'adagrad': Adagrad,
'adadelta': Adadelta,
'adam': Adam,
'adamax': Adamax,
'nadam': Nadam
"""

def size_of_optimisation_space(params):
    space = 1
    for attribute in params.keys():
        if type(attribute) == tuple:
            space*=params[attribute][-1]
        else:
            space*=len(params[attribute])
        
    return space

int(size_of_optimisation_space(opt_params) * 0.05)

48

In [10]:
t = 2
if t:
    print('hello')

hello


### Run Talos

In [8]:
t = ta.Scan(x=X_train,
            y=asarray(y_train).reshape(-1, 1),
            x_val=X_test, 
            y_val=asarray(y_test).reshape(-1, 1),
            model=char_cnn_model_talos,
            round_limit=5,
#             fraction_limit=0.05,
            disable_progress_bar=False,
            params=opt_params)


  0%|          | 0/5 [00:00<?, ?it/s]

Train on 1049 samples, validate on 117 samples
Epoch 1/200
 - 7s - loss: 0.6881 - acc: 0.6120 - val_loss: 0.6766 - val_acc: 0.6410
Epoch 2/200
 - 5s - loss: 0.6560 - acc: 0.6711 - val_loss: 0.7163 - val_acc: 0.7094
Epoch 3/200
 - 5s - loss: 0.6085 - acc: 0.7064 - val_loss: 0.6103 - val_acc: 0.7179
Epoch 4/200
 - 5s - loss: 0.5710 - acc: 0.7302 - val_loss: 0.6210 - val_acc: 0.7009
Epoch 5/200
 - 5s - loss: 0.5385 - acc: 0.7455 - val_loss: 0.6034 - val_acc: 0.7179
Epoch 6/200
 - 5s - loss: 0.6143 - acc: 0.6864 - val_loss: 0.6343 - val_acc: 0.6068
Epoch 7/200
 - 5s - loss: 0.6177 - acc: 0.6540 - val_loss: 0.6067 - val_acc: 0.6752
Epoch 8/200
 - 5s - loss: 0.5405 - acc: 0.7140 - val_loss: 0.6137 - val_acc: 0.6752
Epoch 9/200
 - 5s - loss: 0.5028 - acc: 0.7331 - val_loss: 0.7406 - val_acc: 0.5470
Epoch 10/200
 - 5s - loss: 0.4565 - acc: 0.7255 - val_loss: 0.6977 - val_acc: 0.5812



 20%|██        | 1/5 [00:51<03:26, 51.69s/it]

Train on 1049 samples, validate on 117 samples
Epoch 1/200
 - 5s - loss: 0.7527 - acc: 0.3966 - val_loss: 0.6946 - val_acc: 0.4359
Epoch 2/200
 - 4s - loss: 0.7025 - acc: 0.3661 - val_loss: 0.6948 - val_acc: 0.3932
Epoch 3/200
 - 4s - loss: 0.6934 - acc: 0.3622 - val_loss: 0.6936 - val_acc: 0.3932
Epoch 4/200
 - 4s - loss: 0.6934 - acc: 0.3689 - val_loss: 0.6925 - val_acc: 0.6068
Epoch 5/200
 - 4s - loss: 0.6933 - acc: 0.6378 - val_loss: 0.6919 - val_acc: 0.6068
Epoch 6/200
 - 4s - loss: 0.6935 - acc: 0.6225 - val_loss: 0.6929 - val_acc: 0.6068
Epoch 7/200
 - 4s - loss: 0.6935 - acc: 0.5929 - val_loss: 0.6938 - val_acc: 0.3932
Epoch 8/200
 - 4s - loss: 0.6934 - acc: 0.3966 - val_loss: 0.6939 - val_acc: 0.3932
Epoch 9/200
 - 4s - loss: 0.6934 - acc: 0.4395 - val_loss: 0.6935 - val_acc: 0.3932



 40%|████      | 2/5 [01:25<02:19, 46.36s/it]

...


 60%|██████    | 3/5 [01:51<01:20, 40.19s/it]

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 1049 samples, validate on 117 samples
Epoch 1/200
 - 5s - loss: 0.7475 - acc: 0.5510 - val_loss: 0.6910 - val_acc: 0.6068
Epoch 2/200
 - 4s - loss: 0.6955 - acc: 0.5071 - val_loss: 0.6932 - val_acc: 0.3932
Epoch 3/200
 - 4s - loss: 0.6934 - acc: 0.4280 - val_loss: 0.6931 - val_acc: 0.6154
Epoch 4/200
 - 4s - loss: 0.6934 - acc: 0.3937 - val_loss: 0.6915 - val_acc: 0.6068
Epoch 5/200
 - 4s - loss: 0.6935 - acc: 0.5300 - val_loss: 0.6907 - val_acc: 0.6068
Epoch 6/200
 - 4s - loss: 0.6935 - acc: 0.6282 - val_loss: 0.6925 - val_acc: 0.6068



 80%|████████  | 4/5 [02:15<00:35, 35.47s/it]

Train on 1049 samples, validate on 117 samples
Epoch 1/200
 - 4s - loss: 0.6933 - acc: 0.6368 - val_loss: 0.6910 - val_acc: 0.6068
Epoch 2/200
 - 3s - loss: 0.6927 - acc: 0.6435 - val_loss: 0.6904 - val_acc: 0.6581
Epoch 3/200
 - 3s - loss: 0.6553 - acc: 0.6063 - val_loss: 0.6652 - val_acc: 0.6667
Epoch 4/200
 - 3s - loss: 0.6102 - acc: 0.7102 - val_loss: 0.6530 - val_acc: 0.6752
Epoch 5/200
 - 3s - loss: 0.5999 - acc: 0.7178 - val_loss: 0.6379 - val_acc: 0.6923
Epoch 6/200
 - 3s - loss: 0.5916 - acc: 0.7312 - val_loss: 0.6277 - val_acc: 0.6923
Epoch 7/200
 - 3s - loss: 0.5963 - acc: 0.7293 - val_loss: 0.6475 - val_acc: 0.7265
Epoch 8/200
 - 3s - loss: 0.5943 - acc: 0.6988 - val_loss: 0.6153 - val_acc: 0.6838
Epoch 9/200
 - 3s - loss: 0.5881 - acc: 0.7445 - val_loss: 0.6165 - val_acc: 0.6923
Epoch 10/200
 - 3s - loss: 0.5754 - acc: 0.7359 - val_loss: 0.6153 - val_acc: 0.6923
Epoch 11/200
 - 3s - loss: 0.5654 - acc: 0.7359 - val_loss: 0.6133 - val_acc: 0.7179
Epoch 12/200
 - 3s - loss: 


100%|██████████| 5/5 [03:10<00:00, 41.21s/it]

In [5]:
# Get weights passed to activation function
from talos import Deploy, Predict, Restore
import numpy as np

In [6]:
p = Predict(t)

In [ ]:
p.

In [ ]:
float(p.predict(x[np.newaxis,:,:])) 

In [13]:
results = np.zeros((len(X_test),2))
for i,(y,x) in enumerate(zip(y_test, X_test)):
    results[i,:] = [y, float(p.predict(x[np.newaxis,:,:]))]

In [33]:
import datetime

In [36]:
datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

'2019-07-25_14-35-47'

In [38]:
np.savetxt("foo-" + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + ".csv", 
           results,fmt='%.15f', 
           delimiter=",")

In [19]:
Deploy(scan_object=t, model_name='test_deploy',metric='val_acc')

Deploy package test_deploy have been saved.
data is not 2d, dummy data written instead.


In [16]:
from keras.models import load_model

In [17]:
pwdz

'/home/neil/cloud/habitual_errors_NLP/notebooks'